In [ ]:
%pip install pyarrow

In [1]:
import requests
import pandas as pd

In [2]:
API_KEY = 'c1c0056fd76ba53c273ab417'
BASE_URL = f'https://v6.exchangerate-api.com/v6/{API_KEY}'
DATA = '2025/08/25' # Substitua pela data que você quiser

URL = f'{BASE_URL}/latest/USD'

try:
    response = requests.get(URL)
    response.raise_for_status()  # Isso vai levantar um erro para códigos de status HTTP 4xx/5xx

    data = response.json()
    print(data)

except requests.exceptions.HTTPError as err:
    print(f"Ocorreu um erro HTTP: {err}")
except Exception as err:
    print(f"Ocorreu um erro: {err}")

{'result': 'success', 'documentation': 'https://www.exchangerate-api.com/docs', 'terms_of_use': 'https://www.exchangerate-api.com/terms', 'time_last_update_unix': 1756944001, 'time_last_update_utc': 'Thu, 04 Sep 2025 00:00:01 +0000', 'time_next_update_unix': 1757030401, 'time_next_update_utc': 'Fri, 05 Sep 2025 00:00:01 +0000', 'base_code': 'USD', 'conversion_rates': {'USD': 1, 'AED': 3.6725, 'AFN': 68.8397, 'ALL': 83.7925, 'AMD': 382.3473, 'ANG': 1.79, 'AOA': 918.9807, 'ARS': 1360.17, 'AUD': 1.529, 'AWG': 1.79, 'AZN': 1.6996, 'BAM': 1.6778, 'BBD': 2.0, 'BDT': 121.7066, 'BGN': 1.6779, 'BHD': 0.376, 'BIF': 2976.6347, 'BMD': 1.0, 'BND': 1.2884, 'BOB': 6.9203, 'BRL': 5.4534, 'BSD': 1.0, 'BTN': 88.1043, 'BWP': 13.7059, 'BYN': 3.2664, 'BZD': 2.0, 'CAD': 1.3794, 'CDF': 2886.2969, 'CHF': 0.8041, 'CLP': 973.4898, 'CNY': 7.144, 'COP': 4004.8407, 'CRC': 505.3654, 'CUP': 24.0, 'CVE': 94.5895, 'CZK': 20.9685, 'DJF': 177.721, 'DKK': 6.4004, 'DOP': 63.0241, 'DZD': 130.0, 'EGP': 48.5776, 'ERN': 15.0,

In [3]:
# Salvar o retorno como arquivo JSON na pasta bronze, particionando por timestamp
import os
from datetime import datetime

timestamp = datetime.now().strftime("%Y%m%d")
output_dir = f"bronze/{timestamp}"
os.makedirs(output_dir, exist_ok=True)

with open(f"{output_dir}/retorno_api.json", "w", encoding="utf-8") as f:
    import json
    json.dump(data, f, ensure_ascii=False, indent=4)

In [4]:
# Ler o JSON salvo e salvar como Parquet na pasta silver
import pandas as pd
output_dir = f"bronze/{timestamp}"

with open(f"{output_dir}/retorno_api.json", "r", encoding="utf-8") as f:
    json_data = json.load(f)

# Normaliza o JSON para DataFrame (ajuste conforme necessário)
df = pd.json_normalize(json_data['conversion_rates'])

silver_dir = f"silver/{timestamp}"
os.makedirs(silver_dir, exist_ok=True)
df.to_parquet(f"{silver_dir}/retorno_api.parquet", index=False)

In [12]:
#print(f"O valor do dólar sobre o euro é: {df['EUR'].values[0]}")

# Filtrar e mostrar apenas as moedas da América do Sul
south_american_currencies = ['ARS', 'BOB', 'BRL', 'CLP', 'COP', 'PEN', 'UYU', 'VES', 'GYD', 'PYG', 'SRD']
df_south_america = df[south_american_currencies]
print(df_south_america)

       ARS     BOB     BRL       CLP        COP     PEN      UYU       VES  \
0  1360.17  6.9203  5.4534  973.4898  4004.8407  3.5341  40.0037  151.7627   

        GYD        PYG      SRD  
0  209.0719  7228.9684  38.6265  


In [ ]:

# Instalar dependências da Hugging Face
!pip install transformers accelerate

from transformers import pipeline

# Carregar o modelo gratuito flan-t5-large
llm = pipeline("text2text-generation", model="google/flan-t5-large")

def gerar_insights(df, n_linhas=5):
    """
    Gera insights automáticos sobre o dataset usando LLM gratuita.
    :param df: DataFrame a ser analisado
    :param n_linhas: Quantas linhas do dataset enviar para a LLM
    """
    sample = df.head(n_linhas).to_string()
    prompt = f"""
    Você é um analista de dados. Abaixo estão algumas linhas de um dataset sobre moedas da América do Sul:
    {sample}

    Gere insights relevantes sobre padrões, tendências, oportunidades ou possíveis análises adicionais.
    """
    resposta = llm(prompt, max_length=300, do_sample=True)[0]["generated_text"]
    return resposta

# Exemplo de uso:
insights = gerar_insights(df_south_america)
print("=== Insights gerados pela LLM ===")
print(insights)
